# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ProgettoTS')

Mounted at /content/drive


# Setup and Requirements



In [ ]:
#Do it only first time!
!wget https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
!unzip stanford-corenlp-full-2018-10-05.zip

In [ ]:
!pip install stanza
!pip install textacy
!pip install spacy
!python -m spacy download en_core_web_sm
#NLTK Installation
import nltk
nltk.download('all')
!pip install word2number
!pip install rdflib
!pip install SPARQLWrapper

In [ ]:
import os
os.environ["CORENLP_HOME"] = '/content/drive/MyDrive/ProgettoTS/stanford-corenlp-full-2018-10-05'

# Reading the input file

To upload file from your local drive:

In [ ]:
from google.colab import files
def load_file_locale():
  uploaded = files.upload()
  for name, data in uploaded.items():
    filename = name

  input_f = open(filename, 'r')
  text = input_f.read()
  input_f.close()
  return text

To upload file from your google drive '`ProgettoTS/content`' folder:

In [ ]:
%cd /content/drive/MyDrive/ProgettoTS

/content/drive/.shortcut-targets-by-id/1OhbJiDjKXmlCLStsfCOyS1xyuPNIPYrL/ProgettoTS


In [ ]:
def load_from_colab():
  # Read the input file
  k= input("Insert n. file: ")
  #all files in 'ProgettoTS/content' folder are likely exampleX.txt
  example1 = "content/example"+str(k)+".txt"
  # Open file 
  with open(example1, "r") as file:
      text = file.read()  #file content
  print(example1, "uploaded!")
  return text

In [ ]:

text=load_file_locale()

#Se decommenti la riga successiva scrivi 1 o 2 nella barra che ti uscirà. Sono i file utilizzati durante lo sviluppo del progetto
#text=load_from_colab()


Insert n. file: 2
content/example2.txt uploaded!


'\nSe decommenti la seguente sezione scrivi 1 o 2 nella barra che ti uscirà. Sono i file utilizzati durante lo sviluppo del progetto\n#text=load_from_colab()\n'

In [ ]:
print(text)

Daniel Radcliffe was born at Queen Charlotte's and Chelsea Hospital in Hammersmith, London, England on 23 July 1989, the only child of Marcia Jeannine Gresham and literary agent Alan George Radcliffe. His Jewish mother was born in South Africa, traced her ancestry to Jewish immigrants from Germany, Lithuania, Poland, and Russia, and was raised in the English town of Westcliff-on-Sea. 


In [ ]:

def remove_special_characters(text):
  import re
  # Create a regex pattern to match all characters except letter or number
  pattern = r"[^A-Za-z0-9.,?!;:']+"
  # Remove special characters from the string
  text = re.sub(pattern, ' ', text)
  return text

text=remove_special_characters(text)
print(text)

Daniel Radcliffe was born at Queen Charlotte's and Chelsea Hospital in Hammersmith, London, England on 23 July 1989, the only child of Marcia Jeannine Gresham and literary agent Alan George Radcliffe. His Jewish mother was born in South Africa, traced her ancestry to Jewish immigrants from Germany, Lithuania, Poland, and Russia, and was raised in the English town of Westcliff on Sea. 


Inizialization of the client

In [ ]:
%cd stanford-corenlp-full-2018-10-05

/content/drive/.shortcut-targets-by-id/1OhbJiDjKXmlCLStsfCOyS1xyuPNIPYrL/ProgettoTS/stanford-corenlp-full-2018-10-05


In [ ]:
import stanza
# Import client module
from stanza.server import CoreNLPClient

client = CoreNLPClient(timeout=1500000000000, be_quiet=True, annotators=['openie'], 
endpoint='http://localhost:8100')
client.start()
import time
time.sleep(10)
document = client.annotate(text, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie",
                                 "outputFormat": "json","openie.triple.strict":"true"})

2022-02-26 16:30:47 INFO: Writing properties to tmp file: corenlp_server-2857319608b34d9f.props
2022-02-26 16:30:47 INFO: Starting server with command: java -Xmx5G -cp /content/drive/MyDrive/ProgettoTS/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 8100 -timeout 1500000000000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-2857319608b34d9f.props -annotators openie -preload -outputFormat serialized


# Triples extraction 

In [ ]:

triples = []
for sentence in document['sentences']:
  for triple in sentence['openie']:
    try:
      triples.append({
          'subject': triple['subject'],
          'relation': triple['relation'],
          'object': triple['object']
      })
    except:
      pass


In [ ]:
#Printing triples
for tripla in triples:
  print(tripla)

{'subject': '23 July 1989', 'relation': 'child of', 'object': 'Marcia Jeannine Gresham'}
{'subject': 'Daniel Radcliffe', 'relation': 'was', 'object': "born at Queen Charlotte 's in Hammersmith"}
{'subject': 'Daniel Radcliffe', 'relation': 'was born in', 'object': 'England'}
{'subject': 'Daniel Radcliffe', 'relation': 'was born in', 'object': 'Hammersmith'}
{'subject': 'Hammersmith', 'relation': 'England on', 'object': '23 July 1989'}
{'subject': 'Daniel Radcliffe', 'relation': 'was', 'object': 'born'}
{'subject': 'Daniel Radcliffe', 'relation': 'was born in', 'object': 'London'}
{'subject': 'Daniel Radcliffe', 'relation': 'was born in', 'object': 'England on 23 July 1989'}
{'subject': 'Daniel Radcliffe', 'relation': 'was born at', 'object': 'at Queen Charlotte'}
{'subject': 'His Jewish mother', 'relation': 'was', 'object': 'born'}
{'subject': 'His mother', 'relation': 'was raised in', 'object': 'town'}
{'subject': 'His mother', 'relation': 'was raised in', 'object': 'town of Westcliff'

In [ ]:
#contracted forms
contractions = {
"'m": "am",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"'d": "had",
"'s": "is",
"will've": "have",
"'ve":"have",
"'ll 've":"have",
"'ll be":"be"
}

In [ ]:
from nltk.tokenize import word_tokenize
from word2number import w2n
def dbpedia_query(entity):
  sparql = SPARQLWrapper('https://dbpedia.org/sparql')

  sparql.setQuery( "SELECT * WHERE { ?entity rdfs:label \""+ entity.capitalize()+"\"@en }")
  sparql.setReturnFormat(JSON)
  qres = sparql.query().convert()
  try:
    result = qres['results']['bindings'][0]
    link = result['entity']['value']
  except:
    link=[]
    try:
      entities= word_tokenize(entity)
      for ent in entities:
        try:
          ent=w2n(ent)
        except:
          pass
        sparql.setQuery( "SELECT * WHERE { ?entity rdfs:label \""+ ent.capitalize()+"\"@en }")
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
        result = qres['results']['bindings'][0]
        link.append(result['entity']['value'])
    except:
      pass
  return link


In [ ]:
from nltk.util import print_string
import textacy
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint

f = open("output.txt", "w")
verbPattern = [{"POS": "VERB", "OP": "*"},{"POS": "ADV", "OP": "*"},{"POS": "VERB", "OP": "+"},{"POS": "PART", "OP": "*"}]

counter=0
for tripla in triples:
  counter+=1
  f.write("Triple n°"+ str(counter)+ ' - '+ str(tripla)+"\n\n")
  print("Triple n°"+ str(counter)+ ' - '+ str(tripla)+"\n")

  ###SUBJECT
  f.write('Subject: '+ str(tripla['subject']) +' - '+ str(dbpedia_query(tripla['subject']))+"\n\n")
  print('Subject: '+ str(tripla['subject']) +' - '+ str(dbpedia_query(tripla['subject']))+"\n")
  ###RELATION
  if tripla['relation'] in contractions:
    tripla['relation']=contractions[tripla['relation']] 
  lemma = lemmatizer.lemmatize(tripla['relation'], pos=wordnet.VERB)

  sentence_doc = textacy.make_spacy_doc(tripla['relation'], lang='en_core_web_sm')
  verb_phrases = list(textacy.extract.matches.token_matches(sentence_doc, verbPattern))
  for chunk in verb_phrases:
    lemma = lemmatizer.lemmatize(chunk.text, pos=wordnet.VERB)
    try:
      syn = wordnet.synsets(lemma,pos=wordnet.VERB)[0].definition()
      f.write("Relation: "+ str(lemma) +' - '+ str(syn)+"\n\n")
      print("Relation: "+ str(lemma) +' - '+ str(syn)+"\n")
    except:
      f.write("Relation: "+ str(lemma)+ "No definition found!"+"\n\n")
      print("Relation: "+ str(lemma)+ "No definition found!"+"\n")
  if lemma=="be":
    syn = wordnet.synsets(lemma,pos=wordnet.VERB)[0].definition()
    f.write("Relation: "+ str(lemma)+' - '+ str(syn)+"\n\n")
    print("Relation: "+ str(lemma)+' - '+ str(syn)+"\n")
  ###OBJECT
  f.write('Object: '+ str(tripla['object']) +' - '+ str(dbpedia_query(tripla['object']))+"\n\n")
  print('Object: '+ str(tripla['object']) +' - '+ str(dbpedia_query(tripla['object']))+"\n")
  f.write("-------------------------------------------------------------------------------------------------------------------------------\n\n")
  print("-------------------------------------------------------------------------------------------------------------------------------\n")


f.close()


Triple n°1 - {'subject': '23 July 1989', 'relation': 'child of', 'object': 'Marcia Jeannine Gresham'}

Subject: 23 July 1989 - ['http://dbpedia.org/resource/Category:23', 'http://dbpedia.org/resource/July', 'http://dbpedia.org/resource/1989']

Object: Marcia Jeannine Gresham - ['http://dbpedia.org/resource/Marcia', 'http://dbpedia.org/resource/Jeannine', 'http://dbpedia.org/resource/Gresham']

-------------------------------------------------------------------------------------------------------------------------------

Triple n°2 - {'subject': 'Daniel Radcliffe', 'relation': 'was', 'object': "born at Queen Charlotte 's in Hammersmith"}

Subject: Daniel Radcliffe - http://dbpedia.org/resource/Daniel_radcliffe

Relation: be - have the quality of being; (copula, used with an adjective or a predicate noun)

Object: born at Queen Charlotte 's in Hammersmith - ['http://dbpedia.org/resource/Born', 'http://dbpedia.org/resource/At', 'http://dbpedia.org/resource/Queen', 'http://dbpedia.org/reso